<center><img src="https://storage.googleapis.com/arize-assets/arize-logo-white.jpg" width="200"/></center>

# Object Detection in the Autonomous Vehicle Industry
Walk through how to use Arize for a object detection model using an example dataset.

## 📨 Upload Data to Arize
Upload example data to Arize, this example uses the [Python Pandas method](https://docs.arize.com/arize/sending-data-methods/log-directly-via-sdk-api).

In [ ]:
# Install and import dependencies

!pip install -q arize
from arize.pandas.logger import Client
from arize.utils.types import (
    Schema,
    Environments,
    ModelTypes,
    EmbeddingColumnNames,
    ObjectDetectionColumnNames,
)

import pandas as pd
import numpy as np
from datetime import datetime

### 🌐 Upload Data to Arize: Download Data
Here are sample parquet files that represent the <strong>training</strong> and <strong>production</strong> data of a object detection model designed to identify common object typically seen by autonomous vehicles.

Since object detection models include image data, this dataset includes image vectors to easily visualize embeddings patterns in Arize.

In [ ]:
production_df = pd.read_parquet(
    "https://storage.googleapis.com/arize-assets/fixtures/Industry_Use_Case/autonomous_vehicle_object_detection_production.parquet",
)
training_df = pd.read_parquet(
    "https://storage.googleapis.com/arize-assets/fixtures/Industry_Use_Case/autonomous_vehicle_object_detection_training.parquet",
)

In [ ]:
# Update timestamps to current time

last_ts = max(production_df["prediction_ts"])
now_ts = datetime.timestamp(datetime.now())
delta_ts = now_ts - last_ts

training_df["prediction_ts"] = (training_df["prediction_ts"] + delta_ts).astype(
    float
)
production_df["prediction_ts"] = (
    production_df["prediction_ts"] + delta_ts
).astype(float)


# Reset the index
training_df = training_df.reset_index(drop=True)
production_df = production_df.reset_index(drop=True)

### 🤝 Upload Data to Arize: Create Arize Client
Sign up/login to your Arize account <a href="https://app.arize.com/auth/login">here</a>. Find your <a href="https://docs.arize.com/arize/api-reference/arize.pandas/client">Space and API keys</a>. Copy/paste into the cell below.


In [ ]:
SPACE_ID = "SPACE_ID"  # update value here with your Space ID
API_KEY = "API_KEY"  # update value here with your API key

arize_client = Client(space_id=SPACE_ID, api_key=API_KEY)

In [ ]:
if SPACE_ID == "SPACE_ID" or API_KEY == "API_KEY":
    raise ValueError("❌ CHANGE SPACE_ID AND/OR API_KEY")
else:
    print(
        "✅ Import and Setup Arize Client Done! Now we can start using Arize!"
    )

### 📋 Upload Data to Arize: Define Schema
Create your <a href="https://docs.arize.com/arize/sending-data-to-arize/model-schema-reference">model schema</a>. First, we'll define the features, shap values, and tags. We will use the `ObjectDetectionColumnNames` and `EmbeddingColumnNames` classes from Arize's SDK.

[Here](https://docs.arize.com/arize/sending-data/model-schema-reference#8.-embedding-features-unstructured) is more information about defining embedding features using `EmbeddingColumnNames`.

In [ ]:
tags = ["drift_type"]
embedding_feature_column_names = {
    "image_embedding": EmbeddingColumnNames(
        vector_column_name="image_vector", link_to_data_column_name="url"
    )
}
object_detection_prediction_column_names = ObjectDetectionColumnNames(
    bounding_boxes_coordinates_column_name="prediction_bboxes",
    categories_column_name="prediction_categories",
    scores_column_name="prediction_scores",
)
object_detection_actual_column_names = ObjectDetectionColumnNames(
    bounding_boxes_coordinates_column_name="actual_bboxes",
    categories_column_name="actual_categories",
)

# Define a Schema() object for Arize to pick up data from the correct columns for logging
schema = Schema(
    prediction_id_column_name="prediction_id",
    timestamp_column_name="prediction_ts",
    tag_column_names=tags,
    embedding_feature_column_names=embedding_feature_column_names,
    object_detection_prediction_column_names=object_detection_prediction_column_names,
    object_detection_actual_column_names=object_detection_actual_column_names,
)

### 🪵 Upload Data to Arize: Log Training Data
Define the training schema and log the training data to Arize.

In [ ]:
# Logging Training DataFrame
response = arize_client.log(
    dataframe=training_df,
    schema=schema,
    model_id="object-detection-autonomous-vehicle",
    model_version="1.0",
    model_type=ModelTypes.OBJECT_DETECTION,
    environment=Environments.TRAINING,
)

### 🪵 Upload Data to Arize: Log Production Data
Define the production schema and log the production data to Arize.

In [ ]:
# Logging Production DataFrame
response = arize_client.log(
    dataframe=production_df,
    schema=schema,
    model_id="object-detection-autonomous-vehicle",
    model_version="1.0",
    model_type=ModelTypes.OBJECT_DETECTION,
    environment=Environments.PRODUCTION,
)

## 🏃 Follow 'Success!' Link To Arize
Once you've successfully logged your model to Arize, follow the link to setup monitors, uncover problem areas, and more!

<strong>Note</strong>: It might take a few minutes for all the data to index in Arize, if you don't see all the model data immedieatly, sit back and relax, data is on it's way!

### 🔍 In Arize: Model Setup
Now that we can see our model data in Arize, let's get our model setup with some basic configurations.
* Navigate to the 'Config' tab, click on 'Configure Baseline', and select 'Pre-Production'

<image src="https://storage.googleapis.com/arize-assets/fixtures/Industry_Use_Case/autonomous_baseline.png">

### 🎨In Arize: Embeddings Visualization

To view our data and gain a visual representation of our model performance, navigate to the 'Embeddings' tab.The UMAP visualization will automatically load and surface with the largest discrepancy between production and the baseline (training for this case).
* Click on a [cluster](https://docs.arize.com/arize/embeddings/embedding-and-cluster-analyzer) to surface the images within each cluster


<image src="https://storage.googleapis.com/arize-assets/fixtures/Industry_Use_Case/autonomous_cluster.png" width=850px>

### 🎨In Arize: Embeddings Visualization

As we look through Cluster 20, notice how our production model is 'seeing' a wider variety of wildlife as compared to training. This might indicate an area to focus on as autonomous vehicles explore more rural areas.

* Click on 'Download this cluster' to share this feedback across your teams for fine-tuning and relabeling efforts. Learn more [here](https://docs.arize.com/arize/active-learning-and-fine-tuning/export-data-to-notebook).

## 🚀 Continue Exploring Arize
This tutorial just scratches the surface of what Arize can do. Continue to explore the world of ML Observability with Arize to monitor, troubleshoot, and fine tune your models!

<strong>Recommended Resources:</strong>
* [Arize Community Slack](https://join.slack.com/t/arize-ai/shared_invite/zt-1is2wp3xv-SQgwwszCEeS06Sm1q4xFFw)
* [Arize Documentation](https://docs.arize.com/arize/)
* [ML Observability Course](https://courses.arize.com/)